In [1]:
import tensorflow as tf
tf.__version__

'1.12.0'

In [2]:
import matplotlib
import tqdm

# Tensorflow: GETTING STARTED

Tensorflow is an open-source software library for Machine Intelligence.
(https://www.tensorflow.org/get_started/get_started)

![](https://www.tensorflow.org/images/tensorflow_programming_environment.png)

### Import

In [3]:
import numpy as np
import tensorflow as tf

### what is tensor?

In [4]:
print(3) # a rank 0 tensor; this is a scalar with shape []
print([1. ,2., 3.]) # a rank 1 tensor; this is a vector with shape [3]
print([[1., 2., 3.], [4., 5., 6.]]) # a rank 2 tensor; a matrix with shape [2, 3]
print([[[1., 2., 3.]], [[7., 8., 9.]]]) # a rank 3 tensor with shape [2, 1, 3]

3
[1.0, 2.0, 3.0]
[[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]
[[[1.0, 2.0, 3.0]], [[7.0, 8.0, 9.0]]]


### The Computational Graph
You might think of TensorFlow Core programs as consisting of two discrete sections:

1. Building the computational graph.
2. Running the computational graph.

### tf constants

- 말그대로 상수. 한 번 주어진 값은 변하지 않음.
- 초기화(initializer)가 필요 없음

In [5]:
# create two nodes
node1 = tf.constant(3.0, tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1)
print(node2)

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(), dtype=float32)


### Session
- C++ 백엔드와 파이썬을 연결하여 CPU/GPU 자원을 사용해 계산 수행
- graph 정보를 캐시하여 같은 연산을 효율적으로 반복할 수 있음
- CPU/GPU 자원을 사용하므로, 계산하지 않을 때에는 release하는 습관을 들이자 (`sess.close()`)

In [6]:
# create a session & evaluate the two nodes
sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


In [7]:
# build more nodes with operations
node3 = tf.add(node1, node2)
print("node3: ", node3)
print("sess.run(node3): ",sess.run(node3))

node3:  Tensor("Add:0", shape=(), dtype=float32)
sess.run(node3):  7.0


### tf placeholders

- 그래프에 임의의 외부 데이터를 입력하기 위해 사용

In [8]:
# create placeholders to parametrize our graph
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  # + provides a shortcut for tf.add(a, b)

![](https://www.tensorflow.org/images/getting_started_adder.png)

In [9]:
print(sess.run(adder_node, feed_dict={a: 3, b:4.5}))
print(sess.run(adder_node, feed_dict={a: [1,3,7], b: [2, 4,1]}))

7.5
[3. 7. 8.]


![](https://www.tensorflow.org/images/getting_started_triple.png)

In [10]:
add_and_triple = adder_node * 3.
print(sess.run(add_and_triple, {a: 3, b:4.5}))

22.5


### tf variables

- 모델의 파라미터, 학습 대상

In [11]:
# a simple linear model
W = tf.Variable([.3], tf.float32)
b = tf.Variable([-.3], tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b

★ Before you call `tf.Variable`, you should initialize the variables

In [12]:
# initialize all variables
init = tf.global_variables_initializer()
sess.run(init)

In [13]:
# evaluate the output
print(sess.run(linear_model, feed_dict={x:[1,2,3,4]}))

[0.         0.3        0.6        0.90000004]


### tf loss function

모델이 주어진 데이터에 얼마나 들어맞는지 가늠하려면 손실함수가 필요함

대표적인 손실함수
- regression model: mean-squared error(MSE)
- classification model: cross-entropy

In [14]:
train_x = [1,2,3,4]
train_y = [0,-1,-2,-3]

In [15]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print(sess.run([y,linear_model,squared_deltas,loss], feed_dict={x:train_x, y:train_y}))

[array([ 0., -1., -2., -3.], dtype=float32), array([0.        , 0.3       , 0.6       , 0.90000004], dtype=float32), array([ 0.       ,  1.6899998,  6.7599993, 15.210001 ], dtype=float32), 23.66]


손으로 최적해를 찾을 수도 있다.

In [16]:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x:train_x, y:train_y}))

0.0


그렇지만, 차원이 크고 관계가 복잡하다면...?

데이터를 잘 적합하는 최적의 w, b를 찾기 위한 학습 과정이 필요하다.

In [17]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [18]:
# training
sess.run(init) # reset values to incorrect defaults.

In [19]:
for i in range(1000):
    for mx, my in zip(train_x, train_y):
        sess.run(train, feed_dict={x:[mx], y:[my]})

In [20]:
# evaluate training accuracy
curr_W, curr_b, curr_loss  = sess.run([W, b, loss], {x:train_x, y:train_y})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [-0.99999845] b: [0.999995] loss: 1.7067237e-11


## Summary

### 1. Building a graph
- 입력 데이터: `tf.placeholder` 정의
- 모델 파라미터: `tf.Variable` 정의하기
- 손실 함수: `loss` 계산식 정의
- 학습 알고리즘: `loss`를 최소화하는 데 사용할 `optimizer` 정의

### 2. Running the graph
- 변수 초기화: `tf.global_variables_initializer()`
- 학습 loop: 반복적으로 학습 데이터를 그래프에 대입하여 loss를 최소화

# Visualize it with TensorBoard

`FileWriter`를 사용해 그래프를 저장하자.

In [21]:
tf.reset_default_graph()
a = tf.constant(2)
b = tf.constant(3)
x = tf.add(a, b)
writer = tf.summary.FileWriter('./graphs', tf.get_default_graph())
with tf.Session() as sess:
    # writer = tf.summary.FileWriter('./graphs', sess.graph) 
    print(sess.run(x))
writer.close() # close the writer when you’re done using it

5


저장한 그래프를 브라우저에서 직접 확인하기 위해 아래의 내용을 명령 프롬프트에서 실행하자. (포트 숫자는 마음대로 설정 가능하다.)


```
tensorboard --logdir="./graphs" --port 6006
```

브라우저에서 http://localhost:6006/ 에 접속해보자.